In [1]:
pwd

'/Users/Kelly/galvanize/week4/Federal-Election-Funding'

In [ ]:
'''
get file in jupyter cell
%load filename.py

save to file from jupyter cell (will overwrite)
%%writefile filename.py
'''

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [35]:
# %load build_results_df.py
results_data_76to2018 = pd.read_csv('/Users/Kelly/galvanize/week4/Federal-Election-Funding/data/results-from-Dataverse/house1976-2018.csv', sep=',', header=0)
results_data_80to18 = results_data_76to2018[results_data_76to2018['year']>1979]

In [36]:
results_data_80to18

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
2436,1980,Alabama,AL,1,63,41,US House,1,gen,NaN,False,Steve Smith,libertarian,False,total,6130,117221,False,20171005
2437,1980,Alabama,AL,1,63,41,US House,1,gen,NaN,False,NaN,NaN,True,total,2,117221,False,20171005
2438,1980,Alabama,AL,1,63,41,US House,1,gen,NaN,False,Jack Edwards,republican,False,total,111089,117221,False,20171005
2439,1980,Alabama,AL,1,63,41,US House,2,gen,NaN,False,Adam Hand,libertarian,False,total,4103,172962,False,20171005
2440,1980,Alabama,AL,1,63,41,US House,2,gen,NaN,False,Cecil Wyatt,democrat,False,total,63447,172962,False,20171005
2441,1980,Alabama,AL,1,63,41,US House,2,gen,NaN,False,Bill Dickinson,republican,False,total,104796,172962,False,20171005
2442,1980,Alabama,AL,1,63,41,US House,2,gen,NaN,False,Jerome Couch,statesman,False,total,616,172962,False,20171005
2443,1980,Alabama,AL,1,63,41,US House,3,gen,NaN,False,NaN,NaN,True,total,1561,109215,False,20171005
2444,1980,Alabama,AL,1,63,41,US House,3,gen,NaN,False,Bill Nichols,democrat,False,total,107654,109215,False,20171005
2445,1980,Alabama,AL,1,63,41,US House,4,gen,NaN,False,A.J. Killingsworth,statesman,False,total,2721,132086,False,20171005


In [20]:
results_data_76to2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29701 entries, 0 to 29700
Data columns (total 19 columns):
year              29701 non-null int64
state             29701 non-null object
state_po          29701 non-null object
state_fips        29701 non-null int64
state_cen         29701 non-null int64
state_ic          29701 non-null int64
office            29701 non-null object
district          29701 non-null int64
stage             29671 non-null object
runoff            8727 non-null object
special           29701 non-null bool
candidate         27568 non-null object
party             26261 non-null object
writein           29701 non-null bool
mode              29701 non-null object
candidatevotes    29701 non-null int64
totalvotes        29701 non-null int64
unofficial        29701 non-null bool
version           29701 non-null int64
dtypes: bool(3), int64(8), object(8)
memory usage: 3.7+ MB


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().